In [13]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm.notebook import tqdm
target_epsg = 4269  # What the CBSA files use natively

In [39]:
fn_cbsa = '/Volumes/Extreme SSD/energy_communities/raw_input/cb_2023_us_all_20m/cb_2023_us_cbsa_20m.zip'
fn_csa = '/Volumes/Extreme SSD/energy_communities/raw_input/cb_2023_us_all_20m/cb_2023_us_csa_20m.zip'
fn_out_cbsa_to_tract = '/Volumes/Extreme SSD/energy_communities/clean_input/geography/cbsa_to_tract.parquet'

In [20]:
CbsaRaw = gpd.read_file(fn_cbsa)
Cbsa = CbsaRaw.rename(columns={'CBSAFP':'cbsa_fips', 'GEOIDFQ':'cbsa_geoidfq', 'GEOID':'cbsa_geoid', 'NAME':'cbsa_name'})
Cbsa = Cbsa[['cbsa_fips', 'cbsa_geoid', 'cbsa_geoidfq', 'cbsa_name', 'geometry']]
Cbsa = Cbsa.set_geometry('geometry')
Cbsa = Cbsa.to_crs(epsg=target_epsg)
Cbsa.head()

,cbsa_fips,cbsa_geoid,cbsa_geoidfq,cbsa_name,geometry
0,29860,29860,310M700US29860,"Laurel, MS","POLYGON ((-89.40123 31.79686, -89.31656 31.802..."
1,22840,22840,310M700US22840,"Fort Payne, AL","POLYGON ((-86.11889 34.40384, -86.05771 34.475..."
2,47920,47920,310M700US47920,"Washington Court House, OH","POLYGON ((-83.65333 39.71688, -83.25244 39.695..."
3,23060,23060,310M700US23060,"Fort Wayne, IN","POLYGON ((-85.68657 41.17838, -85.65272 41.178..."
4,16340,16340,310M700US16340,"Cedartown, GA","POLYGON ((-85.42107 34.08081, -85.25807 34.079..."


In [21]:
census_dirs = '/Volumes/Extreme SSD/energy_communities/raw_input/census_tract/2023'
list_of_census_files = [ f.path for f in os.scandir(census_dirs) if f.is_dir() ]
print(list_of_census_files[:3])

['/Volumes/Extreme SSD/energy_communities/raw_input/census_tract/2023/tl_2023_04_tract', '/Volumes/Extreme SSD/energy_communities/raw_input/census_tract/2023/tl_2023_02_tract', '/Volumes/Extreme SSD/energy_communities/raw_input/census_tract/2023/tl_2023_01_tract']


In [33]:
intersection_list = []

for census_subfolder in tqdm(list_of_census_files):
    CensusObservationRaw = gpd.read_file(census_subfolder)
    CensusObservation = CensusObservationRaw.rename(columns={'GEOID':'census_geoid', 'GEOIDFQ':'census_geoidfq', 'geometry':'census_geometry'})
    CensusObservation = CensusObservation[['census_geoid', 'census_geoidfq', 'census_geometry']]
    CensusObservation = CensusObservation.set_geometry('census_geometry')
    CensusObservation = CensusObservation.to_crs(epsg=target_epsg)

    Intersection = gpd.sjoin(Cbsa, CensusObservation, how='inner', predicate='intersects')
    IntersectionIdOnly = Intersection[['cbsa_fips', 'cbsa_geoid', 'cbsa_geoidfq', 'cbsa_name', 'census_geoid', 'census_geoidfq']]
    
    if len(Intersection) > 0:
        intersection_list.append(IntersectionIdOnly)
    else:
        pass

  0%|          | 0/56 [00:00<?, ?it/s]

In [34]:
Concat = pd.concat(intersection_list, axis=0, ignore_index=True)
Concat

,cbsa_fips,cbsa_geoid,cbsa_geoidfq,cbsa_name,census_geoid,census_geoidfq
0,41100,41100,310M700US41100,"St. George, UT",04015950101,1400000US04015950101
1,41100,41100,310M700US41100,"St. George, UT",04015950103,1400000US04015950103
2,49740,49740,310M700US49740,"Yuma, AZ",04027980005,1400000US04027980005
3,49740,49740,310M700US49740,"Yuma, AZ",04027011407,1400000US04027011407
4,49740,49740,310M700US49740,"Yuma, AZ",04027011405,1400000US04027011405
...,...,...,...,...,...,...
94060,38660,38660,310M700US38660,"Ponce, PR",72107954901,1400000US72107954901
94061,38660,38660,310M700US38660,"Ponce, PR",72001956500,1400000US72001956500
94062,38660,38660,310M700US38660,"Ponce, PR",72093960100,1400000US72093960100
94063,38660,38660,310M700US38660,"Ponce, PR",72001956400,1400000US72001956400


In [38]:
Concat.duplicated().sum() == 0

True

In [40]:
Concat.to_parquet(fn_out_cbsa_to_tract)